In [1]:
!pip install sbi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidi

In [2]:
import numpy as np
import math

import seaborn as sns

from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel

In [3]:
from sbi import analysis as analysis
from sbi import utils as utils
from sbi.utils import posterior_nn
from sbi.inference import SNPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
from sbi.inference import DirectPosterior

In [4]:
import random
import numpy as np
import torch

def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

In [5]:
from sbi.utils import BoxUniform
import torch
num_dim = 5
prior_theta = BoxUniform(low=-3 * torch.ones(num_dim), high=3 * torch.ones(num_dim))

import torch
from torch.distributions import MultivariateNormal

def simulate_x_given_theta (theta):
  m_theta = torch.tensor([theta[0], theta[1]])
  s_1 = theta[2]**2; s_2 = theta[3]**2
  rho = torch.tanh(theta[4])
  S_theta = torch.tensor([[s_1**2, rho*s_1*s_2], [rho*s_1*s_2, s_2**2]])

  return MultivariateNormal(m_theta, S_theta).sample()

In [6]:
prior, num_parameters, prior_returns_numpy = process_prior(prior_theta)

simulator = process_simulator(simulate_x_given_theta, prior, prior_returns_numpy)

check_sbi_inputs(simulator, prior)

In [7]:
def h_mat (X_data, Y_data):

    gamma = np.median(np.sum(np.abs(X_data-Y_data)**2,axis=-1))**-1
    kernel_XX = rbf_kernel(X_data,X_data, gamma = gamma)
    kernel_YY = rbf_kernel(Y_data,Y_data, gamma = gamma)
    kernel_XY = rbf_kernel(X_data,Y_data, gamma = gamma)
    kernel_YX = rbf_kernel(Y_data,X_data, gamma = gamma)

    return kernel_XX + kernel_YY - kernel_XY - kernel_YX

def EMMD (knn_matrix, h_mat, n_data, k_n):

    h_mat_knn = np.multiply(h_mat, knn_matrix)

    emmd_val = np.sum(h_mat_knn)/np.sqrt(n_data * k_n)

    return (emmd_val)

def samp_var (knn_matrix, h_mat, n_data, k_n):

    double_edge = ((knn_matrix + knn_matrix.T) == 2).astype(int)
    h2_mat = np.square(h_mat)
    h2_mat_term1 = np.multiply(h2_mat, knn_matrix)
    h2_mat_term2 = np.multiply(h2_mat, double_edge)
    S_squared = (np.sum(h2_mat_term1) + np.sum(h2_mat_term2))/(n_data * k_n)

    return np.sqrt(S_squared)

def emmd_test_stat (knn_matrix, X_data, Y_data, n_data, k_n):

    H_mat = h_mat(X_data, Y_data)
    emmd_val = EMMD(knn_matrix, H_mat, n_data, k_n)
    S_val = samp_var(knn_matrix, H_mat, n_data, k_n)
    return np.absolute(emmd_val/S_val)

In [8]:
from scipy.stats import norm

def EMMD_test (theta_test, x_test, posterior, n_test, k_nn, dim_theta):

  x_knn = kneighbors_graph(x_test, k_nn, include_self = False).toarray()

  posterior_samples = []

  for i in range(n_test):
    posterior_samples.append(posterior.sample((1,), x = x_test[i],\
                                              show_progress_bars = False))

  X = np.array(theta_test)
  Y = np.array(posterior_samples).reshape(-1, dim_theta)

  test = emmd_test_stat(x_knn, X, Y, n_test, k_nn) > norm.ppf(0.975)

  return (test)

def test_power (trained_model, n_test, k_nn, dim_theta, n_rep = 100):

  test_out_vec = np.zeros(n_rep)

  for i in range(n_rep):

    theta_test, x_test = simulate_for_sbi(simulator,\
                                        proposal=prior,\
                                        num_simulations=n_test,\
                                        show_progress_bar = False)

    test_out_vec[i] = EMMD_test(theta_test, x_test,\
                               trained_model, n_test = n_test,\
                               k_nn = k_nn, dim_theta = dim_theta)
    print(i)

  return (np.sum(test_out_vec)/n_rep)

In [13]:
num_sim = [10000, 25000, 50000, 75000, 100000]

num_comp = [5, 10, 15, 25]

In [14]:
from IPython.display import display, clear_output
import time


mdn_numsim_power = np.zeros((len(num_comp), len(num_sim)))

for i in range(len(num_sim)):

  set_all_seeds(1)

  theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=num_sim[i])

  for j in range(len(num_comp)):

    SNPE_posterior_mdn = posterior_nn(model = "mdn", num_components=num_comp[j])
    inferer_mdn_SNPE = SNPE(prior=prior, density_estimator=SNPE_posterior_mdn)

    density_estimator = inferer_mdn_SNPE.append_simulations(theta, x).train(force_first_round_loss=True)
    posterior_mdn_SNPE = inferer_mdn_SNPE.build_posterior(density_estimator)

    mdn_numsim_power[j,i] = test_power(posterior_mdn_SNPE, 1000, 50,\
                                       dim_theta = num_dim, n_rep = 100)

    clear_output(wait=True)
    print (num_sim[i], num_comp[j], mdn_numsim_power[j,i])
    time.sleep(5)


100000 25 0.25


In [15]:
import pandas as pd
mdn_numsim_power_df = pd.DataFrame(mdn_numsim_power)
mdn_numsim_power_df.to_csv('mdn_numsim_power.csv')

In [16]:
num_sim = [10000, 25000, 50000, 75000, 100000]

num_transforms = [5]

In [ ]:
from IPython.display import display, clear_output
import time


nsf_numsim_power = np.zeros((len(num_transforms), len(num_sim)))

for i in range(len(num_sim)):

  set_all_seeds(1)

  theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=num_sim[i])

  for j in range(len(num_transforms)):

    SNPE_posterior_nsf = posterior_nn(model = "nsf", num_transforms=num_transforms[j])
    inferer_nsf_SNPE = SNPE(prior=prior, density_estimator=SNPE_posterior_nsf)

    density_estimator = inferer_nsf_SNPE.append_simulations(theta, x).train(force_first_round_loss=True)
    posterior_nsf_SNPE = inferer_nsf_SNPE.build_posterior(density_estimator)

    nsf_numsim_power[j,i] = test_power(posterior_nsf_SNPE, 1000, 50,\
                                       dim_theta = num_dim, n_rep = 100)

    clear_output(wait=True)
    print (num_sim[i], num_transforms[j], nsf_numsim_power[j,i])
    time.sleep(5)

In [ ]:
import pandas as pd
nsf_numsim_power_df = pd.DataFrame(nsf_numsim_power)
nsf_numsim_power_df.to_csv('nsf_numsim_power.csv')